In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/movielens-25m-dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movielens-25m-dataset/ml-25m-README.html
/kaggle/input/movielens-25m-dataset/ml-25m/ml-25m/movies.csv
/kaggle/input/movielens-25m-dataset/ml-25m/ml-25m/ratings.csv
/kaggle/input/movielens-25m-dataset/ml-25m/ml-25m/genome-tags.csv
/kaggle/input/movielens-25m-dataset/ml-25m/ml-25m/README.txt
/kaggle/input/movielens-25m-dataset/ml-25m/ml-25m/genome-scores.csv
/kaggle/input/movielens-25m-dataset/ml-25m/ml-25m/tags.csv
/kaggle/input/movielens-25m-dataset/ml-25m/ml-25m/links.csv


In [5]:
import numpy as np, pandas as pd, scipy.sparse as sp
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
import os, gc

train = pd.read_parquet("/kaggle/input/movielens-25m-recsys/outputs/parquet/train.parquet")
valid = pd.read_parquet("/kaggle/input/movielens-25m-recsys/outputs/parquet/valid_label.parquet")
test  = pd.read_parquet("/kaggle/input/movielens-25m-recsys/outputs/parquet/test_label.parquet")

## 1) Create sparse matrix → Learn latent factors using SVD

In [6]:
n_users = train["u"].max() + 1
n_items = train["i"].max() + 1

R = sp.coo_matrix(
    (train["rating"], (train["u"], train["i"])),
    shape=(n_users, n_items)
).tocsr()

K = 64
u_f, s, vt = svds(R.astype("float32"), k=K)

S = np.diag(s)
U = u_f @ S      # Users embedding vector (n_users, K)
V = vt.T         # Items embedding vector (n_items, K)

# Function to predict how much a user will like an item 
# by taking the dot product of their embedding vectors.
def mf_score(u_idx, i_idx):
    # embedding vector(s) of selected users * embedding vector(s) of selected items
    return (U[u_idx] * V[i_idx]).sum(axis=1)

## 3) TOP K

In [7]:
TOPK = 50
BATCH = 500
OUT_DIR = "/kaggle/working/batch"
os.makedirs(OUT_DIR, exist_ok=True)

# Indices dictioniary if the users' "seen" items 
seen_dict = (
    train.groupby("u")["i"].apply(lambda x: x.values.astype(np.int64)).to_dict()
)

Vt = V.T.astype(np.float32, copy=False)    # (K, n_items)

all_users = np.arange(n_users, dtype=np.int64)
part_files = []
for bi, start in enumerate(range(0, n_users, BATCH)):
    end = min(start + BATCH, n_users)
    batch_users = all_users[start:end]
    Ubatch = U[batch_users].astype(np.float32, copy=False)

    scores = Ubatch @ Vt     # (batch, n_items)

    # Mask as -np.inf using seen_dict
    for j, u in enumerate(batch_users):
        seen_items = seen_dict.get(u, None)
        if seen_items is not None and len(seen_items) > 0:
            scores[j, seen_items] = -np.inf

    idx_topk = np.argpartition(scores, -TOPK, axis=1)[:, -TOPK:]          # indices for the top K items in each user's row (batch, TOPK)
    part_scores = np.take_along_axis(scores, idx_topk, axis=1)            # scores at the idx_topk (batch, TOPK)
    order_in_part = np.argsort(-part_scores, axis=1)                      # "sorting index" of topk scores
    
    topk_idx_sorted = np.take_along_axis(idx_topk, order_in_part, axis=1) # item indices from sorted (batch, TOPK)
    topk_scores = np.take_along_axis(part_scores, order_in_part, axis=1)  # sorted scores (batch, TOPK)

    uu = np.repeat(batch_users, TOPK).astype(np.int64)
    ii = topk_idx_sorted.reshape(-1).astype(np.int64)   # flattened TOPK idx
    ss = topk_scores.reshape(-1).astype(np.float32)     # flattened TOPK scores
    
    df_part = pd.DataFrame({"u": uu, "i": ii, "mf_score": ss})

    part_path = f"{OUT_DIR}/candidates_part_{bi:03d}.parquet"
    df_part.to_parquet(part_path, index=False)
    part_files.append(part_path)

    del scores, idx_topk, part_scores, order_in_part, topk_idx_sorted, topk_scores, df_part, uu, ii, ss
    gc.collect()

print("Baseline candidate files saved:", len(part_files))

Baseline candidate files saved: 75
